In [ ]:
import numpy as np
import pandas as pd
import re
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
def add_datepart(df, fldname, drop=True, time=False):
    """
    Adiciona colunas ao dataframe relacionadas a uma coluna de data especificada.
    
    Parâmetros:
    - df (pd.DataFrame): DataFrame de entrada.
    - fldname (str): Nome da coluna de data no DataFrame.
    - drop (bool): Se verdadeiro, a coluna de data original será removida.
    - time (bool): Se verdadeiro, colunas relacionadas ao tempo (hora, minuto, segundo) serão adicionadas.
    
    Retorna:
    - None: O DataFrame original (df) é modificado no local.
    """
    
    # Garante que a coluna especificada é do tipo datetime
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64
    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    
    # Prefixo para os novos nomes de colunas
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    
    # Atributos que serão extraídos da data
    attr = ['Year', 'Month', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    
    # Se o parâmetro time for verdadeiro, adiciona colunas relacionadas ao tempo
    if time: 
        attr = attr + ['Hour', 'Minute', 'Second']
    
    # Loop através dos atributos e adicione-os ao dataframe
    for n in attr: 
        if n == 'Week': 
            # 'Week' foi depreciado, usando isocalendar().week em vez disso
            df[targ_pre + n] = fld.dt.isocalendar().week
        else:
            df[targ_pre + n] = getattr(fld.dt, n.lower())
    
    # Adiciona uma coluna com o tempo em formato Unix
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    
    # Remove a coluna de data original, se solicitado
    if drop: 
        df.drop(fldname, axis=1, inplace=True)


In [ ]:
# Gerando dados fictícios
# Definindo as datas
date_range = pd.date_range(start='2019-01-01', end='2020-12-31', freq='D')

# Gerando características numéricas
np.random.seed(42)  # Para reprodutibilidade
temperatura = np.random.normal(25, 5, len(date_range))  # Média de 25 graus e desvio padrão de 5
precipitacao = np.random.gamma(2, 10, len(date_range))  # Gamma distribuição para simular precipitação

# Gerando características categóricas
estacoes = ['primavera', 'verão', 'outono', 'inverno']
evento_climatico = ['ensolarado', 'chuvoso', 'nublado']

# Associando as estações aos meses
estacao = np.array(estacoes * (len(date_range) // 4 + 1))[:len(date_range)]
np.random.shuffle(estacao)  # Misturando as estações aleatoriamente
evento = np.random.choice(evento_climatico, len(date_range))

# Gerando a variável alvo fictícia
# Suponha que o consumo de energia é maior em dias frios e em dias chuvosos
consumo_energia = 1000 + (25 - temperatura) * 10 + precipitacao * 2 + np.random.normal(0, 50, len(date_range))

# Criando o DataFrame
df = pd.DataFrame({
    'data': date_range,
    'temperatura': temperatura,
    'precipitacao': precipitacao,
    'estacao': estacao,
    'evento_climatico': evento,
    'consumo_energia': consumo_energia
})

In [ ]:
#X e y
y = df['consumo_energia']
df = df.drop(columns='consumo_energia')

In [ ]:
# Feature engineering temporal
add_datepart(df, 'data', drop=False, time=False)

In [ ]:
df.info()

In [ ]:
# Definindo transformadores

# Identificando automaticamente características numéricas e categóricas
numeric_features = df.select_dtypes(include=['int32','int64', 'float64']).columns.tolist()
categorical_features = df.select_dtypes(include=['object', 'bool']).columns.tolist()

# Definindo os pipelines conforme fornecido
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Transformador de colunas: aplica as transformações acima nas respectivas colunas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
# Combina o pré-processamento com o modelo em um único pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('modelo', RandomForestRegressor())
                          ])

In [ ]:
# CERTIFIQUE-SE QUE O DATAFRAME ESTEJA ORDENADO CRESCENTE PELA COLUNA DE DATA #
## Identifique os índices únicos para cada data
unique_dates = df['data'].drop_duplicates().sort_values()
date_indices = range(len(unique_dates))

In [ ]:
# Definindo a estratégia de validação cruzada respeitando a estrutura temporal
tscv = TimeSeriesSplit(n_splits=5) # aqui será respeitada a estrutura temporal - dados em teste serão sempre em momento futuro aos dados do treino

# Fazemos nossa função na mão para resolver o problema com temporalidade
# Listas para armazenar scores de cada fold
scores = []
for train_date_idx, test_date_idx in tscv.split(date_indices):
    # Converta índices de data em índices de dataframe
    train_idx = df[df['data'].isin(unique_dates.iloc[train_date_idx])].index
    test_idx = df[df['data'].isin(unique_dates.iloc[test_date_idx])].index
    
    # Separe os conjuntos de treinamento e teste usando os índices
    X_train, y_train = df.drop(['data'], axis=1).iloc[train_idx], y.iloc[train_idx]
    X_test, y_test = df.drop(['data'], axis=1).iloc[test_idx], y.iloc[test_idx]
    
    # Treine o pipeline no conjunto de treinamento
    pipeline.fit(X_train, y_train)
    
    # Faça previsões no conjunto de teste
    y_pred = pipeline.predict(X_test)
    
    # Calcule a métrica de erro (por exemplo, MAE)
    mae = mean_absolute_error(y_test, y_pred)
    
    # Adicione o score à lista de scores
    scores.append(mae)


In [ ]:
# Calcule a média e o desvio padrão dos escores
mean_score = np.mean(scores)
std_score = np.std(scores)
print(f"Média de MAE: {mean_score}, Desvio padrão: {std_score}")